In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.max_columns", 100)

### Load raw dataset

In [3]:
RAW_PATH = "../data/raw/BDIR61FL.csv"

df_raw = pd.read_csv(RAW_PATH)  # df_raw = OG copy
df = df_raw.copy()              # df = working copy

### Shape 

In [4]:
df.shape


(17842, 1827)

In [5]:
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17842 entries, 0 to 17841
Columns: 1827 entries, caseid to s541y_3
dtypes: float64(1599), int64(224), object(4)
memory usage: 248.7+ MB


### Null Columns 

In [6]:
null_cols = df.columns[df.isnull().all()]
len(null_cols)


0

### Columns with only ONE unique value

In [7]:
constant_cols = df.columns[df.nunique() <= 1]
len(constant_cols)


390

In [8]:
df.drop(columns=constant_cols, inplace=True)


### Identity Info drop

In [9]:
id_like_cols = [
    "caseid", "hhid", "cluster", "line", "v001", "v002", "v003", "v004", "v005", "v006", "v007","v008","v009"
]

df.drop(columns=id_like_cols, errors="ignore", inplace=True)


### Checkpoint_1

In [10]:
CLEAN_PATH = "../data/processed/01_basic_cleaned.csv"
df.to_csv(CLEAN_PATH, index=False)


### Sanity check

In [11]:
print("Before:", df_raw.shape)
print("After :", df.shape)


Before: (17842, 1827)
After : (17842, 1428)
